In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
model = "gpt-3.5-turbo"

llm = ChatOpenAI(temperature=0, model=model)

d:\WorkSpace\Projects\Case-study\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings


loader = DirectoryLoader("data/", glob="*.txt", show_progress=True, loader_cls=TextLoader, loader_kwargs={"autodetect_encoding": True})
index = VectorstoreIndexCreator(
    embedding=OpenAIEmbeddings(),
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

100%|██████████| 1110/1110 [00:00<00:00, 6048.42it/s]


d:\WorkSpace\Projects\Case-study\venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [4]:
resp = index.query("all the requirements and description of Product Management", llm=llm)

In [5]:
print(resp)

Product Management typically involves working closely with various teams within an organization to develop, launch, and manage products or services. Some of the key responsibilities and requirements for a Product Manager include:

1. Collaborating with product development personnel on product specifications such as design, color, or packaging.
2. Using sales forecasting and strategic planning to ensure the sale and profitability of products, analyzing business developments, and monitoring market trends.
3. Negotiating contracts with vendors or distributors to manage product distribution.
4. Coordinating promotional activities or trade shows to market products or services.
5. Developing and implementing technical project management tools.
6. Directing team activities, establishing task priorities, scheduling work assignments, and providing guidance.
7. Reporting project plans, progress, and results to managers or other personnel.
8. Analyzing project data to determine specifications or 

In [6]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel 
from typing import List 

class Plan(BaseModel):
    plan: List[List[str]]

parser = PydanticOutputParser(pydantic_object=Plan)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"plan": {"items": {"items": {"type": "string"}, "type": "array"}, "title": "Plan", "type": "array"}}, "required": ["plan"]}
```


In [23]:
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain

prompt_1 = PromptTemplate(
    template='''
    Create a list of {num_keywords} ATS
keywords associated with the given role. The keywords should be as distinct 
and unique as possible, and one element in the list of keyword should not be the extension 
of another keyword. Do not have duplicates and do not repeat any keyword. 
Use the given description for the role to get a better understanding 
of the role.

Role: {role}

Description: {description}
    ''', 
    input_variables=['num_keywords', 'role', 'description']
)

prompt_2 = PromptTemplate(
    template=''' 
    From the given list of ATS keywords and role, cluster 
    the keywords with the exact same meaning in the same list. Use as many keywords as possible
    and each nested list should represent distinct meaning. 
    For the given list of keywords, create 'Plan' which is a list of 'Item' where 
    each Item represents a list of keywords with exact same meaning. 
    for example, given list [l1, l2, l3, l4] if l1 and l2 have same exact meaning and 
    l3 and l4 also have same exact meaning, the generated list will be Plan: [Item_1: [l1, l2], Item_2: [l3, l4]]. 
    Instead of eliminating the keyword, merge the keywords with same meaning 
    in the same list. If and only if a keyword have a distinct meaning than all the other keywords, it can be alone 
    in the list.
    
    Role: {role}
    
    Keywords: {keywords}
    
    Format Instructions: {format_instructions}
    ''', 
    input_variables=['role', 'keywords'], 
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

prompt_3 = PromptTemplate(
    template=''' 
    From the given nested list of ATS keywords, with the given role. Make sure that
    the ATS keywords in a same nested list must have exact same meaning. If the ATS keywords 
    in an Item does not have same meaning, create a new Item and append it to the Plan. 
    Use as many keywords as possible
    and each Item should represent distinct meaning.
    For example, given nested list - Plan: [Item_1: [l1, l2], Item_2: [l3, l4], Item_3: [l5]] 
    if l1 and l2 does not have same 
    meaning, the refined list will be - Plan: [Item_1: [l1], Item_2: [l3, l4], Item_3: [l5], Item_4: [l2]]
    Instead of eliminating the keyword, merge the keywords with same meaning 
    in the same list. If and only if a keyword have a distinct meaning than all the other keywords, it can be alone 
    in the list.
    
    The input and output format is same. 
    
    Role: {role}
    
    Keywords: {curated_keywords}
    
    Format Instructions: {format_instructions}
    ''',
    input_variables=['role', 'curated_keywords'],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain_1 = LLMChain(llm=llm, prompt=prompt_1, 
                   output_key="keywords")
chain_2 = LLMChain(llm=llm, prompt=prompt_2, 
                   output_key="curated_keywords")
chain_3 = LLMChain(llm=llm, prompt=prompt_3, 
                   output_key="refined_keywords")

overall_chain = SequentialChain(
    chains=[chain_1, chain_2, chain_3], 
    input_variables=['role', 'num_keywords', 'description'], 
    output_variables=['keywords', 'curated_keywords', 'refined_keywords'],
    verbose=True
)

In [24]:
resp_chain = overall_chain({"role": 'product management', "num_keywords": 200, "description": resp})



> Entering new SequentialChain chain...

> Finished chain.


In [25]:
print(resp_chain['curated_keywords'])

{
    "plan": [
        ["Product development collaboration", "Product feature development collaboration"],
        ["Sales forecasting", "Product feature sales forecasting"],
        ["Strategic planning", "Product feature strategic planning"],
        ["Profitability analysis", "Product feature profitability analysis"],
        ["Market trend monitoring", "Product feature market trend monitoring"],
        ["Vendor contract negotiation", "Product feature vendor contract negotiation"],
        ["Distribution management", "Product feature distribution management"],
        ["Promotional activities coordination", "Product feature promotional activities coordination"],
        ["Trade show marketing", "Product feature trade show marketing"],
        ["Technical project management", "Product feature technical project management"],
        ["Task prioritization", "Product feature task prioritization"],
        ["Work assignment scheduling", "Product feature work assignment scheduling"],
  

In [26]:
print(resp_chain['refined_keywords'])

{
    "plan": [
        ["Product development collaboration", "Product feature development collaboration"],
        ["Sales forecasting", "Product feature sales forecasting"],
        ["Strategic planning", "Product feature strategic planning"],
        ["Profitability analysis", "Product feature profitability analysis"],
        ["Market trend monitoring", "Product feature market trend monitoring"],
        ["Vendor contract negotiation", "Product feature vendor contract negotiation"],
        ["Distribution management", "Product feature distribution management"],
        ["Promotional activities coordination", "Product feature promotional activities coordination"],
        ["Trade show marketing", "Product feature trade show marketing"],
        ["Technical project management", "Product feature technical project management"],
        ["Task prioritization", "Product feature task prioritization"],
        ["Work assignment scheduling", "Product feature work assignment scheduling"],
  

In [27]:
print(resp_chain['keywords'])

1. Product development collaboration
2. Sales forecasting
3. Strategic planning
4. Profitability analysis
5. Market trend monitoring
6. Vendor contract negotiation
7. Distribution management
8. Promotional activities coordination
9. Trade show marketing
10. Technical project management
11. Task prioritization
12. Work assignment scheduling
13. Guidance provision
14. Project reporting
15. Data analysis
16. Specification determination
17. Requirement analysis
18. Data validation
19. Statistical testing
20. Customer needs understanding
21. Customer satisfaction assurance
22. Bachelor's degree
23. Communication skills
24. Analytical skills
25. Problem-solving skills
26. Business background
27. Marketing background
28. Qualifications assessment
29. Work experience evaluation
30. Team leadership
31. Product launch planning
32. Product management strategy
33. Product lifecycle management
34. Product portfolio management
35. Product roadmap development
36. Product feature prioritization
37. Pr

In [28]:
import json

with open("result.json", "w") as file:
    file.write(parser.parse(resp_chain['refined_keywords']).model_dump_json())